In [0]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [0]:
#seed(1)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/Data Mining/ML_APPROACH/Data/10percent/')

In [0]:
#Import training data
train=pd.read_csv('1_train_set.csv')
train.dropna(inplace=True)
train=train.reset_index(drop=True)

In [0]:
#Import test data
test=pd.read_csv('1_test_set.csv')
test.dropna(inplace=True)
test=test.reset_index(drop=True)

In [0]:
train.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'booking_month', 'booking_day', 'ci_month', 'co_month',
       'ci_day', 'co_day', 'duration', 'remaining_days', 'is_weekend', 'dest0',
       'dest1', 'dest2', 'dest3', 'dest4', 'dest5', 'dest6', 'dest7', 'dest8',
       'dest9', 'dest10', 'dest11', 'dest12', 'dest13', 'dest14', 'dest15',
       'dest16', 'dest17', 'dest18', 'dest19', 'dest20', 'dest21', 'dest22',
       'dest23', 'dest24', 'dest25', 'dest26', 'dest27', 'dest28', 'dest29',
       'dest30', 'dest31', 'dest32', 'dest33', 'dest34', 'dest35', 'dest36',
       'dest37', 'dest38', 'dest39'],
      dtype='object')

In [0]:
#List of categorical features
cat_cols=['site_name', 'posa_continent', 
        'user_location_country',
       'user_location_region', 'user_location_city',
        'channel', 'hotel_continent', 
       'hotel_country', 'hotel_market',
       'booking_month', 'booking_day', 
       'ci_month', 'co_month',
       'ci_day', 'co_day']

In [0]:
#List of numerical features
num_cols=['orig_destination_distance', 'is_mobile', 'is_package',
       'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'is_booking', 'cnt','duration', 'remaining_days', #'is_weekend', 
       'dest0',
       'dest1', 'dest2', 'dest3', 'dest4', 'dest5', 'dest6', 'dest7', 'dest8',
       'dest9', 'dest10', 'dest11', 'dest12', 'dest13', 'dest14', 'dest15',
       'dest16', 'dest17', 'dest18', 'dest19', 'dest20', 'dest21', 'dest22',
       'dest23', 'dest24', 'dest25', 'dest26', 'dest27', 'dest28', 'dest29',
       'dest30', 'dest31', 'dest32', 'dest33', 'dest34', 'dest35', 'dest36',
       'dest37', 'dest38', 'dest39']

In [0]:
#Target variable for training and test sets
y=train.hotel_cluster
y_test_reserved=test.hotel_cluster

In [0]:
import keras

Using TensorFlow backend.


In [0]:
y_train = keras.utils.to_categorical(y,num_classes=100)
y_test= keras.utils.to_categorical(y_test_reserved,num_classes=100)

In [0]:
#Drop target from training and test sets
train.drop(columns=['hotel_cluster','user_id'], inplace=True)
test.drop(columns=['hotel_cluster','user_id'], inplace=True)

In [0]:
#One hot encode categorical variables
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [0]:
enc.fit(train[cat_cols])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

In [0]:
#Create sparse matrices
from scipy.sparse import csr_matrix, hstack

In [0]:
XC =enc.transform(train[cat_cols])
XN=csr_matrix(train[num_cols].values)
X=csr_matrix(hstack((XN,XC)))

In [0]:
XC_test =enc.transform(test[cat_cols])
XN_test=csr_matrix(test[num_cols].values)
X_test=csr_matrix(hstack((XN_test,XC_test)))

In [0]:
#Standardize variables
from sklearn.preprocessing import StandardScaler

In [0]:
scaler=StandardScaler(with_mean=False).fit(X)

In [0]:
X_train=scaler.transform(X)

In [0]:
X_test=scaler.transform(X_test)

In [0]:
X_train

<2996331x38766 sparse matrix of type '<class 'numpy.float64'>'
	with 184651507 stored elements in Compressed Sparse Row format>

In [0]:
#Imort libraries for building MLP
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [0]:
#Build MLP
model=Sequential()

In [0]:
model.add(Dense(200, activation='relu', input_dim=38766))
model.add(Dropout(0.5))
model.add(Dense(100 , activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))

In [0]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [0]:
import time
start=time.time()

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
MLP_model=model.fit(X_train, y_train,
          epochs=20,
          batch_size=10000,
          validation_split=0.001)

Train on 2993334 samples, validate on 2997 samples
Epoch 1/10
2993334/2993334 [==============================] - 520s 174us/step - loss: 3.6872 - accuracy: 0.1095 - val_loss: 3.4429 - val_accuracy: 0.1245
Epoch 2/10
2993334/2993334 [==============================] - 519s 173us/step - loss: 3.5355 - accuracy: 0.1237 - val_loss: 3.4148 - val_accuracy: 0.1325
Epoch 3/10
2993334/2993334 [==============================] - 518s 173us/step - loss: 3.5287 - accuracy: 0.1257 - val_loss: 3.4019 - val_accuracy: 0.1355
Epoch 4/10
2993334/2993334 [==============================] - 520s 174us/step - loss: 3.5390 - accuracy: 0.1259 - val_loss: 3.4117 - val_accuracy: 0.1308
Epoch 5/10
2993334/2993334 [==============================] - 522s 174us/step - loss: 3.5574 - accuracy: 0.1255 - val_loss: 3.4126 - val_accuracy: 0.1361
Epoch 6/10
2993334/2993334 [==============================] - 522s 174us/step - loss: 3.5759 - accuracy: 0.1249 - val_loss: 3.4275 - val_accuracy: 0.1348
Epoch 7/10
2993334/299333

In [0]:
end=time.time()
print(end-start)
#1 h plus 20-30 min

5504.040465831757


In [0]:
os.chdir("/content/drive/My Drive")

In [0]:
#save the model
from sklearn.externals import joblib
filename = 'finalized_model.sav'
joblib.dump(MLP_model, filename)  

['finalized_model.sav']

In [0]:
result = joblib.load(filename)

In [0]:
result.model.evaluate(X_test, y_test)

749128/749128 [==============================] - 194s 259us/step


[3.4464611086627097, 0.13357530534267426]

In [0]:
#Make the prediction for test set
preds=result.model.predict(X_test, verbose=1)

749128/749128 [==============================] - 158s 211us/step


In [0]:
#Define the function for assessing accuracy
def map5eval(preds, actual):
    predicted = preds.argsort(axis=1)[:,-np.arange(5)]
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
    metric /= actual.shape[0]
    return 'MAP@5', metric

In [0]:
map5eval(preds,y_test_reserved)

('MAP@5', 0.1272814080726747)

In [0]:
#Save probabilities for test set
os.chdir('/content/drive/My Drive/Data Mining/ML_APPROACH/Predicted_Proba/')
pd.DataFrame(preds).to_csv('pred_proba_MLP.csv', index=False)